In [1]:
%load_ext autotime 

import scanpy as sc
import scvi
import torch
from rich import print


import scvelo as scv
import loompy
import scanpy
import igraph
import glob, os
import pandas as pd
import os
import re
import subprocess
from collections import Counter
import numpy as np
from shutil import rmtree

import anndata

import h5py
from shutil import copyfile

time: 5.63 s (started: 2024-12-16 08:51:02 +01:00)


In [2]:
ipath = "../BD_results/quantify_bam_output_multi_correct/BD_Rhapsody_expression/"
raw = "correct_quantification_v2_test_raw.h5ad"
analyzed = "correct_quantification_v2_test_analyzed.h5ad"

time: 275 μs (started: 2024-12-16 08:51:08 +01:00)


In [3]:
print("Last run with scvi-tools version:", scvi.__version__)
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

Last run with scvi-tools version: 1.2.0

scanpy==1.10.3
scvi==1.2.0
torch==2.5.1+cu124
scvelo==0.3.3.dev12+g22b6e7e
loompy==3.0.7
scanpy==1.10.3
igraph==0.11.6
pandas==2.2.3
re==2.2.1
numpy==1.26.4
anndata==0.10.8
h5py==3.12.1

time: 7.57 ms (started: 2024-12-16 08:51:08 +01:00)


In [4]:
adata = scanpy.read_10x_mtx( ipath )

time: 25.6 s (started: 2024-12-16 08:51:08 +01:00)


In [5]:
adata

AnnData object with n_obs × n_vars = 112327 × 102327
    var: 'gene_ids', 'feature_types'

time: 1.93 ms (started: 2024-12-16 08:51:33 +01:00)


In [6]:
# Extract gene names and identify suffixes
gene_names = adata.var.index.tolist()
spliced_genes = [name for name in gene_names if not name.endswith(('_unspliced','_unspliced_antisense', '_antisense', '_ext'))]
unspliced_genes = [name for name in gene_names if name.endswith('_unspliced')]
antisense_genes = [name for name in gene_names if name.endswith('_antisense')]
unspl_antisens_genes =  [name for name in gene_names if name.endswith('_unspliced_antisense')]
ext_genes = [name for name in gene_names if name.endswith('_ext')]
unspl_antisens_genes[1:10]

['Srcin1_unspliced_antisense',
 'Arhgef10_unspliced_antisense',
 'Pot1a_unspliced_antisense',
 'Rorb_unspliced_antisense',
 '4933409G03Rik_unspliced_antisense',
 'Nlrc4_unspliced_antisense',
 'Phf1_unspliced_antisense',
 'Gm36431_unspliced_antisense',
 'Atg10_unspliced_antisense']

time: 35.9 ms (started: 2024-12-16 08:51:33 +01:00)


In [7]:
# Remove suffixes for consistency
spliced_genes_clean = spliced_genes
antisense_genes_clean = [name[:-10] for name in antisense_genes]
unspliced_genes_clean = [name[:-10] for name in unspliced_genes]  # Remove '_unspliced'
unspl_antisens_genes_clean = [name[:-20] for name in unspl_antisens_genes]  # Remove '_unspliced_antisense'
unspl_antisens_genes_clean[1:10]

['Srcin1',
 'Arhgef10',
 'Pot1a',
 'Rorb',
 '4933409G03Rik',
 'Nlrc4',
 'Phf1',
 'Gm36431',
 'Atg10']

time: 6.99 ms (started: 2024-12-16 08:51:33 +01:00)


In [8]:
print(antisense_genes_clean[1:10])
antisense_genes_clean = [name for name in antisense_genes_clean if not name.endswith(('_unspliced' ))]
antisense_genes_clean[1:10]

[
    'Srcin1_unspliced',
    'Cdh3',
    'Arhgef10_unspliced',
    'Rnf121',
    'Exosc7',
    'Pot1a_unspliced',
    'Agpat5',
    'Rorb_unspliced',
    'Gm15631'
]

['Rnf121',
 'Exosc7',
 'Agpat5',
 'Gm15631',
 'Olfr133',
 'Tbpl1',
 'Tnni3k',
 'Hnrnpab',
 'A230087F16Rik']

time: 8.44 ms (started: 2024-12-16 08:51:33 +01:00)


In [9]:
# Create a unified gene set
all_genes = sorted(set(spliced_genes_clean + unspliced_genes_clean + antisense_genes_clean + unspl_antisens_genes_clean))
len(all_genes)

32200

time: 12.5 ms (started: 2024-12-16 08:51:33 +01:00)


In [10]:
# Identify _ext genes
ext_genes = [name for name in gene_names if name.endswith('_ext')]

# Extract _ext expression data
ext_mask = adata.var.index.isin(ext_genes)
ext_data = pd.DataFrame(
    adata[:, ext_mask].X.toarray(),
    index=adata.obs_names,
    columns=[name[:-4] for name in ext_genes]  # Remove '_ext' suffix for column names
)

# Add _ext data to adata.obs
for col in ext_data.columns:
    adata.obs[col] = ext_data[col] # Convert sparse data to dense for obs
adata

AnnData object with n_obs × n_vars = 112327 × 102327
    obs: 'JH584302.1', 'SampleTag08_mm|stAbO', 'GL456239.1', 'SampleTag04_mm|stAbO', 'SampleTag05_mm|stAbO', 'SampleTag07_mm|stAbO', 'GL456370.1', 'SampleTag02_mm|stAbO', 'GL456378.1', 'MU069435.1', 'GL456382.1', 'GL456396.1', 'GL456360.1', 'GL456381.1', 'GL456359.1', 'GL456390.1', 'SampleTag06_mm|stAbO', 'GL456366.1', 'GL456385.1', 'GL456387.1', 'JH584301.1', 'SampleTag10_mm|stAbO', 'GL456368.1', 'GL456233.2', 'GL456394.1', 'GL456379.1', 'GL456372.1', 'GL456392.1', 'JH584300.1', 'GL456383.1', 'MU069434.1', 'GL456367.1', 'SampleTag09_mm|stAbO', 'phiX174', 'GL456389.1', 'SampleTag11_mm|stAbO', 'SampleTag01_mm|stAbO', 'SampleTag12_mm|stAbO', 'SampleTag03_mm|stAbO'
    var: 'gene_ids', 'feature_types'

time: 23 ms (started: 2024-12-16 08:51:37 +01:00)


In [11]:
# Remove _ext genes from the main data
non_ext_mask = ~ext_mask
adata._inplace_subset_var(non_ext_mask)

time: 354 ms (started: 2024-12-16 08:51:41 +01:00)


In [12]:
adata

AnnData object with n_obs × n_vars = 112327 × 102288
    obs: 'JH584302.1', 'SampleTag08_mm|stAbO', 'GL456239.1', 'SampleTag04_mm|stAbO', 'SampleTag05_mm|stAbO', 'SampleTag07_mm|stAbO', 'GL456370.1', 'SampleTag02_mm|stAbO', 'GL456378.1', 'MU069435.1', 'GL456382.1', 'GL456396.1', 'GL456360.1', 'GL456381.1', 'GL456359.1', 'GL456390.1', 'SampleTag06_mm|stAbO', 'GL456366.1', 'GL456385.1', 'GL456387.1', 'JH584301.1', 'SampleTag10_mm|stAbO', 'GL456368.1', 'GL456233.2', 'GL456394.1', 'GL456379.1', 'GL456372.1', 'GL456392.1', 'JH584300.1', 'GL456383.1', 'MU069434.1', 'GL456367.1', 'SampleTag09_mm|stAbO', 'phiX174', 'GL456389.1', 'SampleTag11_mm|stAbO', 'SampleTag01_mm|stAbO', 'SampleTag12_mm|stAbO', 'SampleTag03_mm|stAbO'
    var: 'gene_ids', 'feature_types'

time: 1.76 ms (started: 2024-12-16 08:51:42 +01:00)


In [13]:
from scipy.sparse import csr_matrix
# Create layers for unspliced and antisense with aligned genes
def align_layer(adata, genes, real_name, layer_name):
    aligned_data = np.zeros((adata.n_obs, len(all_genes)))
    gene_to_idx = {gene: i for i, gene in enumerate(all_genes)}
    for i, gene in enumerate(genes):
        if gene in all_genes:
            aligned_data[:, gene_to_idx[gene]] = adata[:, real_name[i]].X.toarray().flatten()
    return csr_matrix(aligned_data)


time: 543 μs (started: 2024-12-16 08:51:47 +01:00)


In [14]:
all_genes[0:10]

['0610005C13Rik',
 '0610006L08Rik',
 '0610009B22Rik',
 '0610009E02Rik',
 '0610009L18Rik',
 '0610010K14Rik',
 '0610012D04Rik',
 '0610012G03Rik',
 '0610025J13Rik',
 '0610030E20Rik']

time: 1.58 ms (started: 2024-12-16 08:51:47 +01:00)


In [15]:
spliced_genes_clean[0:10]

['Pcdhb8',
 'Mcrip1',
 'Ighv2-9',
 'Mmgt2',
 'Fam184a',
 'Orai1',
 'Gm8013',
 'ENSMUSG00000120264',
 'Arpc5l',
 'Pom121']

time: 1.33 ms (started: 2024-12-16 08:51:48 +01:00)


In [16]:
antisense_genes[0:10]

['Zbtb48_unspliced_antisense',
 'Srcin1_unspliced_antisense',
 'Cdh3_antisense',
 'Arhgef10_unspliced_antisense',
 'Rnf121_antisense',
 'Exosc7_antisense',
 'Pot1a_unspliced_antisense',
 'Agpat5_antisense',
 'Rorb_unspliced_antisense',
 'Gm15631_antisense']

time: 1.42 ms (started: 2024-12-16 08:51:50 +01:00)


In [17]:
spliced = align_layer(adata, spliced_genes_clean, spliced_genes, 'antisense')
spliced

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 105926808 stored elements and shape (112327, 32200)>

time: 1min 13s (started: 2024-12-16 08:51:58 +01:00)


In [18]:
# Build the new AnnData object
new_adata = anndata.AnnData(
    X=spliced,
    obs=adata.obs.copy(),
    var=pd.DataFrame(index=all_genes)
)

time: 12.1 ms (started: 2024-12-16 08:53:15 +01:00)


In [19]:
new_adata

AnnData object with n_obs × n_vars = 112327 × 32200
    obs: 'JH584302.1', 'SampleTag08_mm|stAbO', 'GL456239.1', 'SampleTag04_mm|stAbO', 'SampleTag05_mm|stAbO', 'SampleTag07_mm|stAbO', 'GL456370.1', 'SampleTag02_mm|stAbO', 'GL456378.1', 'MU069435.1', 'GL456382.1', 'GL456396.1', 'GL456360.1', 'GL456381.1', 'GL456359.1', 'GL456390.1', 'SampleTag06_mm|stAbO', 'GL456366.1', 'GL456385.1', 'GL456387.1', 'JH584301.1', 'SampleTag10_mm|stAbO', 'GL456368.1', 'GL456233.2', 'GL456394.1', 'GL456379.1', 'GL456372.1', 'GL456392.1', 'JH584300.1', 'GL456383.1', 'MU069434.1', 'GL456367.1', 'SampleTag09_mm|stAbO', 'phiX174', 'GL456389.1', 'SampleTag11_mm|stAbO', 'SampleTag01_mm|stAbO', 'SampleTag12_mm|stAbO', 'SampleTag03_mm|stAbO'

time: 1.68 ms (started: 2024-12-16 08:53:17 +01:00)


In [20]:
new_adata.layers['unspliced'] =align_layer(adata, unspliced_genes_clean, unspliced_genes, 'antisense')

time: 1min 15s (started: 2024-12-16 08:53:20 +01:00)


In [21]:
new_adata.layers['antisense'] =align_layer(adata, antisense_genes_clean, antisense_genes, 'antisense')

time: 1min 7s (started: 2024-12-16 08:54:36 +01:00)


In [22]:
#unspl_antisens_genes_clean = [name[:-20] for name in unspl_antisens_genes]
new_adata.layers['antisense_unspliced'] = align_layer(adata, unspl_antisens_genes_clean, unspl_antisens_genes, 'antisense')

time: 56.6 s (started: 2024-12-16 08:55:43 +01:00)


In [23]:
new_adata.layers['spliced'] = new_adata.X

time: 392 μs (started: 2024-12-16 08:56:40 +01:00)


In [24]:
new_adata

AnnData object with n_obs × n_vars = 112327 × 32200
    obs: 'JH584302.1', 'SampleTag08_mm|stAbO', 'GL456239.1', 'SampleTag04_mm|stAbO', 'SampleTag05_mm|stAbO', 'SampleTag07_mm|stAbO', 'GL456370.1', 'SampleTag02_mm|stAbO', 'GL456378.1', 'MU069435.1', 'GL456382.1', 'GL456396.1', 'GL456360.1', 'GL456381.1', 'GL456359.1', 'GL456390.1', 'SampleTag06_mm|stAbO', 'GL456366.1', 'GL456385.1', 'GL456387.1', 'JH584301.1', 'SampleTag10_mm|stAbO', 'GL456368.1', 'GL456233.2', 'GL456394.1', 'GL456379.1', 'GL456372.1', 'GL456392.1', 'JH584300.1', 'GL456383.1', 'MU069434.1', 'GL456367.1', 'SampleTag09_mm|stAbO', 'phiX174', 'GL456389.1', 'SampleTag11_mm|stAbO', 'SampleTag01_mm|stAbO', 'SampleTag12_mm|stAbO', 'SampleTag03_mm|stAbO'
    layers: 'unspliced', 'antisense', 'antisense_unspliced', 'spliced'

time: 2.34 ms (started: 2024-12-16 08:56:40 +01:00)


In [25]:
new_adata.write( raw )
!ls -lhrt {raw}

-rw-r--r-- 1 stefanl jyuan 3.3G Dec 16 08:56 correct_quantification_v2_test_raw.h5ad
time: 3.13 s (started: 2024-12-16 08:56:40 +01:00)
